In [19]:
## load SCE object
sce_R_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_d52_cells_sce.Rds"
sce = readRDS(sce_R_filename)
sce

class: SingleCellExperiment 
dim: 32738 79801 
metadata(0):
assays(1): counts
rownames(32738): MIR1302-10 FAM138A ... AC002321.2 AC002321.1
rowData names(0):
colnames(79801): CCAGCGAGTTACTGAC-1-64 ACGCAGCGTTCGCTAA-1-45 ...
  CTTAGGAAGATTACCC-1-13 TCAACGACATAGAAAC-1-39
colData names(21): batch celltype ... treatment index
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [20]:
### Filter SCE object
ncol(sce)
######
# consider only untreated cells
sce_filt = sce[,sce$treatment == 'NONE']
######
# consider only donors with > 10 cells
N=10
donors_keep = names(table(sce_filt$donor_id)[table(sce_filt$donor_id)>N])
sce_filt = sce_filt[, sce_filt$donor_id %in% donors_keep]  
ncol(sce_filt)

[1] 79801

Warning message in NSBS(i, x, exact = exact, strict.upper.bound = !allow.append, :
“subscript is an array, passing it thru as.vector() first”

[1] 54217

In [21]:
# filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_10nn.csv"
# filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_5nn.csv"
filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_20nn.csv"
df_clusters = read.csv(filename, row.names = 1)

In [22]:
head(df_clusters)

cell,cluster,donor_id,ncells,pseudo_cell
CCAGCGAGTTACTGAC-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
GATCTAGCATCCAACA-1-65,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
TGGCCAGCAAGCTGTT-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
GCTGCGACAAGGACAC-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
ACATGGTGTTATCACG-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
GTTACAGCACAGAGGT-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1


In [23]:
length(unique(df_clusters$pseudo_cell))

[1] 1013

In [24]:
length(unique(df_clusters$donor_id))

[1] 173

In [25]:
##### normalisation and log

In [26]:
sce_filt = scater::logNormCounts(sce_filt)
sce_filt

class: SingleCellExperiment 
dim: 32738 54217 
metadata(0):
assays(2): counts logcounts
rownames(32738): MIR1302-10 FAM138A ... AC002321.2 AC002321.1
rowData names(0):
colnames(54217): CCAGCGAGTTACTGAC-1-64 ACGTCAAGTCTAGCCG-1-51 ...
  CCCAATCCACAGGAGT-1-25 CTTAGGAAGATTACCC-1-13
colData names(21): batch celltype ... treatment index
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [27]:
##### (mean) aggregation

In [28]:
mean = matrix(0, nrow = nrow(sce_filt), ncol = length(unique(df_clusters$pseudo_cell)))
for (i in 1:length(unique(df_clusters$pseudo_cell))){
    pseudocell = unique(df_clusters$pseudo_cell)[i]
    cells = df_clusters[df_clusters$pseudo_cell == pseudocell,"cell"]
    if (length(cells)==1){
        mean[,i] = as.matrix(logcounts(sce_filt))[,cells]
    }
    else{
        mat = as.matrix(logcounts(sce_filt))[,cells]
        mean[,i] = rowMeans(mat)
    }  
}

In [29]:
mean[1:5,1:5]

0.00000000,0,0,0.01026937,0
0.00000000,0,0,0.00000000,0
0.00000000,0,0,0.00000000,0
0.01005067,0,0,0.00000000,0
0.00000000,0,0,0.00000000,0


In [30]:
rownames(mean) = rownames(sce_filt)
colnames(mean) = unique(df_clusters$pseudo_cell)

In [31]:
# write.csv(mean, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_10nn_means.csv")
# write.csv(mean, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_5nn_means.csv")
write.csv(mean, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_20nn_means.csv")

In [32]:
dim(mean)

[1] 32738  1013

In [33]:
##### PCs computation

In [34]:
pcs = prcomp(t(mean))
head(pcs$x)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC1004,PC1005,PC1006,PC1007,PC1008,PC1009,PC1010,PC1011,PC1012,PC1013
HPSI0513i-veve_2-cluster1,-1.17523125,0.84892215,-0.08146942,0.15655695,-0.6967274,0.28227213,-0.05731137,0.02687786,-0.02134157,0.10126158,...,-0.0380371445,-0.013026875,0.021552168,0.012350756,-0.0122232423,-4.897995e-03,-0.004438731,0.0101493398,-0.045066088,-3.184952e-15
HPSI0513i-veve_2-cluster2,0.84146079,0.22260747,0.84807600,-0.07730129,-0.7728794,0.13413230,0.12760102,0.19813195,0.15956085,-0.26819218,...,0.0003896857,0.004858092,-0.010901712,0.009847001,0.0040027973,-1.451151e-02,0.002304555,0.0058711976,-0.007868976,-3.195361e-15
HPSI0513i-veve_2-cluster3,-0.63925555,-0.31068403,0.01348205,-0.22818228,0.4359577,-0.06800035,0.30661798,0.14596256,-0.11628088,-0.04343866,...,-0.0132364111,-0.008627547,0.010072140,0.015795353,-0.0159749592,-3.684294e-03,-0.018438656,0.0140914393,-0.008501964,-3.646389e-15
HPSI0513i-veve_2-cluster4,0.67626037,-0.04684597,0.25972304,-0.00993146,-0.3681330,0.11597488,0.01237422,-0.07470260,0.23524685,-0.34115176,...,-0.0158844297,0.006075425,0.034507976,0.021268194,0.0177197597,-2.590899e-02,-0.003574705,0.0003941334,0.025279101,-3.438222e-15
HPSI0513i-veve_2-cluster5,-1.08851900,-0.54299994,-0.66155683,0.01496061,0.4611878,-0.29070602,-0.04436174,-0.09526424,0.25365940,-0.30596934,...,-0.0173799232,0.036185689,-0.003629423,-0.016869085,-0.0002670184,-5.756575e-03,0.009373425,0.0075149462,0.011009543,-3.330669e-15
HPSI0513i-veve_2-cluster6,-0.01467169,-0.23621582,-0.53758700,-0.14241322,0.5468148,-0.23856872,0.13824077,-0.23273385,0.21031312,-0.12296327,...,-0.0187629626,-0.014601022,-0.006108957,0.005000708,0.0046744487,5.541614e-05,-0.002709230,-0.0101321198,-0.021644834,-3.150258e-15


In [35]:
# write.csv(pcs$x[,1:500], "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_10nn_mean_500pcs.csv")
# write.csv(pcs$x[,1:500], "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_5nn_mean_500pcs.csv")
write.csv(pcs$x[,1:500], "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_20nn_mean_500pcs.csv")

In [36]:
sample_df = data.frame(genotype_individual_id = gsub("-cluster.*","",rownames(pcs$x)),
                       phenotype_sample_id = rownames(pcs$x))
tail(sample_df)

,genotype_individual_id,phenotype_sample_id
1008,HPSI1113i-ieki_3,HPSI1113i-ieki_3-cluster12
1009,HPSI1113i-ieki_3,HPSI1113i-ieki_3-cluster13
1010,HPSI0414i-ceik_1,HPSI0414i-ceik_1-cluster1
1011,HPSI0414i-ceik_1,HPSI0414i-ceik_1-cluster2
1012,HPSI0114i-vass_1,HPSI0114i-vass_1-cluster1
1013,HPSI0114i-vass_1,HPSI0114i-vass_1-cluster2


In [37]:
# write.csv(sample_df, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_10nn_sample_mapping.csv", quote = F)
# write.csv(sample_df, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_5nn_sample_mapping.csv", quote = F)
write.csv(sample_df, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_20nn_sample_mapping.csv", quote = F)